In [ ]:
import joblib
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import util
from deep_translator import MyMemoryTranslator
import ast

In [ ]:

modelo_LR = joblib.load('models/modelo_sentimientos.pkl')
modelo_embedding = joblib.load('models/modelo_embedding.pkl')
modelo_recomendaciones = joblib.load('models/modelo_recomendaciones.pkl')
modelo_gliner = joblib.load('models/modelo_gliner.pkl')


In [ ]:
juegos_dataset = pd.read_csv('./databases/juegos_procesados.csv')
libros_dataset = pd.read_csv('./databases/libros_procesados.csv')
peliculas_dataset = pd.read_csv('./databases/peliculas_procesados.csv')

juegos_dataset['embeddings'] = juegos_dataset['description'].apply(lambda x: modelo_recomendaciones.encode(x))
libros_dataset['embeddings'] = libros_dataset['description'].apply(lambda x: modelo_recomendaciones.encode(x))
peliculas_dataset['embeddings'] = peliculas_dataset['description'].apply(lambda x: modelo_recomendaciones.encode(x))


In [ ]:
def traducir(query):
    translated = MyMemoryTranslator(source='spanish', target='english').translate(query)
    return translated

In [ ]:
def busqueda_peliculas(consulta):
    #convierto los valores de las columnas en listas
    peliculas_dataset['actors'] = peliculas_dataset['actors'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    peliculas_dataset['categories'] = peliculas_dataset['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    consulta_usuario = consulta
    labels = ['Name','Genre','Year']
    entidades_consulta = modelo_gliner.predict_entities(traducir(consulta_usuario),labels,threshold=0.6)
    names = []
    years = []
    genres = []
    for entidad in entidades_consulta:
        if entidad['label'] == 'Name':
            if entidad['text'].lower() == 'i':
                continue
            names.append(entidad['text'].lower())
        if entidad['label'] == 'Year':
            years.append(entidad['text'].lower())
        if entidad['label'] == 'Genre':
            genres.append(entidad['text'].lower())

    candidates = peliculas_dataset.copy()
    # Filtrar por nombres en 'director' o 'actors'
    if len(names) > 0:
        candidates = candidates = peliculas_dataset[
        peliculas_dataset.apply(lambda row: any(name in [actor for actor in row['actors']] or name in [director for director in row['director']] for name in names), axis=1)]
     # Filtrar por género en 'categories' 
    if len(genres) > 0:
        candidates = candidates[
            candidates.apply(lambda row: any(genre in [category for category in row['categories']] for genre in genres), axis=1)
        ]

    # Filtrar por años en 'years' 
    if len(years) > 0:
        candidates = candidates[
            candidates.apply(lambda row: any(str(year) in str(row['year']) for year in years), axis=1)
        ]
    
    # Si no hay candidates hace busqueda semantica en todo el dataset
    if len(candidates) == 0:
        candidates = peliculas_dataset.copy()
    
    user_embedding = modelo_recomendaciones.encode(traducir(consulta_usuario),convert_to_tensor=True)
    # Calculamos el coseno para determinar los mejores resultados
    coseno = util.cos_sim(user_embedding, candidates['embeddings'].tolist())[0]
    index = coseno.argsort(descending=True)[:3].tolist()
    recomendaciones = [candidates['title'].iloc[i] for i in index]
    return recomendaciones


In [ ]:
def busqueda_libros(consulta):
    #convierto los valores de las columnas en listas
    libros_dataset['categories'] = libros_dataset['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    consulta_usuario = consulta
    labels = ['Genre']
    entidades_consulta = modelo_gliner.predict_entities(traducir(consulta_usuario),labels,threshold=0.6)
    genres = []
    for entidad in entidades_consulta:

        if entidad['label'] == 'Genre':
            genres.append(entidad['text'].lower())

    candidates = libros_dataset.copy()

     # Filtrar por género en 'categories' 
    if len(genres) > 0:
        candidates = candidates[
            candidates.apply(lambda row: any(genre in [category for category in row['categories']] for genre in genres), axis=1)
        ]

    if len(candidates) == 0:
        candidates = libros_dataset.copy()

    # Si no hay candidates hace busqueda semantica en todo el dataset
    user_embedding = modelo_recomendaciones.encode(traducir(consulta_usuario),convert_to_tensor=True)
    # Calculamos el coseno para determinar los mejores resultados
    coseno = util.cos_sim(user_embedding, candidates['embeddings'].tolist())[0]
    index = coseno.argsort(descending=True)[:3].tolist()
    recomendaciones = [candidates['title'].iloc[i] for i in index]
    return recomendaciones

In [ ]:
def busqueda_juegos(consulta):
    #convierto los valores de las columnas en listas
    juegos_dataset['directors'] = juegos_dataset['directors'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    juegos_dataset['categories'] = juegos_dataset['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    consulta_usuario = consulta
    labels = ['Name','Genre','Year','Numplayers']
    entidades_consulta = modelo_gliner.predict_entities(traducir(consulta_usuario),labels,threshold=0.6)
    names = []
    years = []
    genres = []
    numplayers = 1
    for entidad in entidades_consulta:
        if entidad['label'] == 'Name':
            if entidad['text'].lower() == 'i':
                continue
            names.append(entidad['text'].lower())
        if entidad['label'] == 'Year':
            years.append(entidad['text'].lower())
        if entidad['label'] == 'Genre':
            genres.append(entidad['text'].lower())
        if entidad['label'] == 'Numplayers':
            numplayers = int(entidad['text'][0])

    candidates = juegos_dataset.copy()
    # Filtrar por nombres en 'director' o 'actors'
    if len(names) > 0:
        candidates = candidates = juegos_dataset[
        juegos_dataset.apply(lambda row: any(name in [director for director in row['directors']] for name in names), axis=1)]
     # Filtrar por género en 'categories' (solo si hay géneros extraídos)
    if len(genres) > 0:
        candidates = candidates[
            candidates.apply(lambda row: any(genre in [category for category in row['categories']] for genre in genres), axis=1)
        ]

    # Filtrar por años en 'years' (solo si hay años extraídos)
    if len(years) > 0:
        candidates = candidates[
            candidates.apply(lambda row: any(str(year) in str(row['year']) for year in years), axis=1)
        ]
    #Filtrar por la cantidad de jugadores
    candidates.apply(lambda row: row['minplayers'] <= numplayers <= row['maxplayers'], axis=1)
    
    # Si no hay candidates hace busqueda semantica en todo el dataset
    if len(candidates) == 0:
        candidates = juegos_dataset.copy()

    user_embedding = modelo_recomendaciones.encode(traducir(consulta_usuario),convert_to_tensor=True)
    # Calculamos el coseno para determinar los mejores resultados
    coseno = util.cos_sim(user_embedding, candidates['embeddings'].tolist())[0]
    index = coseno.argsort(descending=True)[:3].tolist()
    recomendaciones = [candidates['title'].iloc[i] for i in index]
    return recomendaciones
  

In [ ]:
def predict_sentiment(text):
    text_vectorized = modelo_embedding.encode([text])

    sentiment_class = modelo_LR.predict(text_vectorized)[0]
    
    labels = {0: "Melancolico", 1: "Ni fu Ni fa", 2: "Alegre"}
    
    sentiment_label = labels[sentiment_class]
    
    return sentiment_label

In [ ]:
initial_prompt = widgets.HTML(
    description='Asistente:',
    value='Hola, ¿Cómo estás?',
    layout=widgets.Layout(width='100%'),
)

followup_prompt = widgets.HTML(
    description='Asistente:',
    value='Veo que te estas sintiendo reemplazarPorSentimiento. ¿Quieres leer un libro(1), ver una pelicula(2) o jugar un juego(3)?',
    layout=widgets.Layout(width='100%')
)

user_initial_input = widgets.Text(
    placeholder='Escriba una respuesta',
    layout=widgets.Layout(width='100%'),
    disabled=False
)

user_followup_input = widgets.Text(
    placeholder='Escribe 1 para libro, 2 para pelicula o 3 para juego',
    layout=widgets.Layout(width='100%'),
    disabled=False
)
user_secondfollowup_input = widgets.Text(
    placeholder='Escribe el tema que te interesa',
    layout=widgets.Layout(width='100%'),
    disabled=False
)

second_followup_prompt = widgets.HTML(
    description='Asistente:',
    layout=widgets.Layout(width='100%')
)

button_initial = widgets.Button(description='Responder')
button_followup = widgets.Button(description='Responder')
button_secondfollowup = widgets.Button(description='Responder')
output = widgets.Output()

recommendation_display = widgets.Textarea(
    description='Asistente:',
    layout=widgets.Layout(width='100%'),
    disabled=True
)



# Callback for initial response
def on_initial_button_clicked(b):
    if(user_initial_input.value == ''):
        return
    with output:
        button_initial.layout.display = 'none'
        output.clear_output()
        sentiment = predict_sentiment(user_initial_input.value)
        followup_prompt.value = followup_prompt.value.replace('reemplazarPorSentimiento', sentiment.lower())
        display(followup_prompt)
        display(user_followup_input)
        display(button_followup)

# Callback for follow-up response
def on_followup_button_clicked(b):
    with output:
        prompt = user_followup_input.value
        if(prompt == '1'):
            second_followup_prompt.value = f"Genial! vamos a buscar libro, sobre que quieres leer?"
        elif(prompt == '2'):
            second_followup_prompt.value = f"Genial! vamos a buscar una película que te guste, sobre que quieres ver?"
        elif(prompt == '3'):
            second_followup_prompt.value = f"Genial! vamos a buscar un juego que te guste, que tema quieres abordar?"
        else:
            user_followup_input.placeholder = 'Por favor ingrese 1, 2 o 3'
            user_followup_input.value = ''
            return
        button_followup.layout.display = 'none'

        display(second_followup_prompt)
        display(user_secondfollowup_input)
        display(button_secondfollowup)

# Callback for follow-up response
def on_secondfollowup_button_clicked(b):
    if(user_secondfollowup_input.value == ''):
        return
    with output:
        button_followup.layout.display = 'none'
        prompt = user_secondfollowup_input.value
        tipo = user_followup_input.value
        if(tipo == '1'):
            recomendaciones = busqueda_libros(prompt)
        elif(tipo == '2'):
            recomendaciones = busqueda_peliculas(prompt)
        elif(tipo == '3'):
            recomendaciones = busqueda_juegos(prompt)

        recommendation_display.value = f"Encontre estas recomendaciones para ti:\n"
        for i, recomendacion in enumerate(recomendaciones):
            recommendation_display.value += f"{i+1}. {recomendacion},\n"

        if len(recomendaciones) == 0:
            recommendation_display.value = "No encontré recomendaciones para esa descripción"
        
        display(recommendation_display)

# Bind buttons to callbacks
button_initial.on_click(on_initial_button_clicked)
button_followup.on_click(on_followup_button_clicked)
button_secondfollowup.on_click(on_secondfollowup_button_clicked)

# Display initial prompt and input
display(initial_prompt)
display(user_initial_input)
display(button_initial)
display(output)
